In [2]:
import pandas as pd 
from sqlalchemy import text
import sys
import os

# Add the parent directory to the system path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.db.database import get_engine

# --- Define the Main SQL Query ---
# This query joins all necessary tables and calculates key metrics like revenue, cost, and profit per order item.
sql_query = """
SELECT
    c.customer_id,
    c.city,
    o.order_id,
    o.order_date,
    o.shipped_date,
    p.product_name,
    cat.category_name,
    od.quantity,
    od.unit_price,
    od.discount,
    --- Key Calculation: Estimated Item Cost
    --- It is assumed that the cost is 70% of the unit price for demonstration purposes.
    (od.unit_price * 0.7) AS estimated_item_cost,

    --- KPI Calculation: Groos Revenue and Profit per Item
    --- Gross Revenue (takes quantity and discount into account)
    (od.quantity * od.unit_price * (1 - od.discount)) AS item_gross_revenue,

    -- Total Cost for the item line 
    (od.quantity * (od.unit_price * 0.7)) AS item_total_cost,

    -- Profit for the item line
    ((od.quantity * od.unit_price * (1 - od.discount)) - (od.quantity * (od.unit_price * 0.7))) AS item_profit
FROM
    order_details od
JOIN
    orders o ON od.order_id = o.order_id
JOIN
    customers c ON o.customer_id = c.customer_id
JOIN
    products p ON od.product_id = p.product_id
JOIN
    categories cat ON p.category_id = cat.category_id;
"""

# --- Execute the Query and Load Data into a DataFrame ---
print("Connecting to the database and executing the query...")
try: 
    engine = get_engine()
    # Read the SQL query into a pandas DataFrame
    advanced_df = pd.read_sql_query(sql=text(sql_query), con=engine)
    print("Query executed successfully. Data loaded into DataFrame.")

    # Dispose of the engine connection pool
    engine.dispose()

except Exception as e:
    print(f"An error occurred while connecting to the database or executing the query: {e}")
    advanced_df = pd.DataFrame()  # Create an empty DataFrame in case of error

# --- Initial Data Inspection and Type Verification ---
if not advanced_df.empty:
    print("\n--- DataFrame Info ---")
    advanced_df.info() # To check data types and non-null counts

    print("\n--- First 5 Rows of the DataFrame ---")
    display(advanced_df.head())  

    # 1.1 Requirement: Ensure order_date is a datetime type
    # Verify the data type of 'order_date' column
    if pd.api.types.is_datetime64_any_dtype(advanced_df['order_date']):
        print("\nVerification successful: 'order_date' is of type datetime.")
    else:
        print("\nWarning: 'order_date' is NOT of type datetime. Manual conversion may be required.")

Connecting to the database and executing the query...
Database connection established successfully.
Query executed successfully. Data loaded into DataFrame.

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_id          2155 non-null   object 
 1   city                 2155 non-null   object 
 2   order_id             2155 non-null   int64  
 3   order_date           2155 non-null   object 
 4   shipped_date         2082 non-null   object 
 5   product_name         2155 non-null   object 
 6   category_name        2155 non-null   object 
 7   quantity             2155 non-null   int64  
 8   unit_price           2155 non-null   float64
 9   discount             2155 non-null   float64
 10  estimated_item_cost  2155 non-null   float64
 11  item_gross_revenue   2155 non-null   float64
 12  item_to

,customer_id,city,order_id,order_date,shipped_date,product_name,category_name,quantity,unit_price,discount,estimated_item_cost,item_gross_revenue,item_total_cost,item_profit
0,VINET,Reims,10248,1996-07-04,1996-07-16,Queso Cabrales,Dairy Products,12,14.0,0.0,9.800000,168.000000,117.600000,50.400000
1,VINET,Reims,10248,1996-07-04,1996-07-16,Singaporean Hokkien Fried Mee,Grains/Cereals,10,9.8,0.0,6.860000,98.000002,68.600001,29.400001
2,VINET,Reims,10248,1996-07-04,1996-07-16,Mozzarella di Giovanni,Dairy Products,5,34.8,0.0,24.359999,173.999996,121.799997,52.199999
3,TOMSP,Münster,10249,1996-07-05,1996-07-10,Tofu,Produce,9,18.6,0.0,13.020000,167.400003,117.180002,50.220001
4,TOMSP,Münster,10249,1996-07-05,1996-07-10,Manjimup Dried Apples,Produce,40,42.4,0.0,29.680001,1696.000061,1187.200043,508.800018


In [3]:
# --- Data Type Conversion --- 
if not advanced_df.empty:
    print("\n--- Converting 'order_date' and 'shipped_date' to datetime ---")

    # Convert 'order_date' and 'shipped_date' to datetime, handling errors  
    advanced_df['order_date'] = pd.to_datetime(advanced_df['order_date'], errors='coerce')
    advanced_df['shipped_date'] = pd.to_datetime(advanced_df['shipped_date'], errors='coerce')

    print("\n--- Verifying Data Types After Conversion ---")
    advanced_df.info() 

    print("\n--- First 5 Rows of the DataFrame After Conversion ---")
    display(advanced_df.head())

else:
    print("DataFrame is empty. Skipping data type conversion.")


--- Converting 'order_date' and 'shipped_date' to datetime ---

--- Verifying Data Types After Conversion ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   customer_id          2155 non-null   object        
 1   city                 2155 non-null   object        
 2   order_id             2155 non-null   int64         
 3   order_date           2155 non-null   datetime64[ns]
 4   shipped_date         2082 non-null   datetime64[ns]
 5   product_name         2155 non-null   object        
 6   category_name        2155 non-null   object        
 7   quantity             2155 non-null   int64         
 8   unit_price           2155 non-null   float64       
 9   discount             2155 non-null   float64       
 10  estimated_item_cost  2155 non-null   float64       
 11  item_gross_revenue   2155 non-null  

,customer_id,city,order_id,order_date,shipped_date,product_name,category_name,quantity,unit_price,discount,estimated_item_cost,item_gross_revenue,item_total_cost,item_profit
0,VINET,Reims,10248,1996-07-04,1996-07-16,Queso Cabrales,Dairy Products,12,14.0,0.0,9.800000,168.000000,117.600000,50.400000
1,VINET,Reims,10248,1996-07-04,1996-07-16,Singaporean Hokkien Fried Mee,Grains/Cereals,10,9.8,0.0,6.860000,98.000002,68.600001,29.400001
2,VINET,Reims,10248,1996-07-04,1996-07-16,Mozzarella di Giovanni,Dairy Products,5,34.8,0.0,24.359999,173.999996,121.799997,52.199999
3,TOMSP,Münster,10249,1996-07-05,1996-07-10,Tofu,Produce,9,18.6,0.0,13.020000,167.400003,117.180002,50.220001
4,TOMSP,Münster,10249,1996-07-05,1996-07-10,Manjimup Dried Apples,Produce,40,42.4,0.0,29.680001,1696.000061,1187.200043,508.800018


In [4]:
# --- Calculate Shipping Time ---

# Calculate shipping time in days and add as a new column

if 'order_date' in advanced_df.columns and 'shipped_date' in advanced_df.columns:
    print("Calculating the difference between 'shipped_date' and 'order_date'...")

    # Subtract 'order_date' from 'shipped_date' to get shipping time
    advanced_df['shipping_days'] = (advanced_df['shipped_date'] - advanced_df['order_date']).dt.days

    print("New column 'shipping_days' added successfully.")

    # --- Verification ---
    print("\n--- Displaying relevant columns to verify the calculation ---")

    # Display relevant columns to verify the calculation
    verification_columns = ['order_date', 'shipped_date', 'shipping_days']
    display(advanced_df[verification_columns].head(10))

    print("\n--- Verifying the data type of the new column ---")
    # Check the data type of the new column
    advanced_df.info()
else: 
    print("Date columns not found, skipping shipping time calculation.")

Calculating the difference between 'shipped_date' and 'order_date'...
New column 'shipping_days' added successfully.

--- Displaying relevant columns to verify the calculation ---


,order_date,shipped_date,shipping_days
0,1996-07-04,1996-07-16,12.0
1,1996-07-04,1996-07-16,12.0
2,1996-07-04,1996-07-16,12.0
3,1996-07-05,1996-07-10,5.0
4,1996-07-05,1996-07-10,5.0
5,1996-07-08,1996-07-12,4.0
6,1996-07-08,1996-07-12,4.0
7,1996-07-08,1996-07-12,4.0
8,1996-07-08,1996-07-15,7.0
9,1996-07-08,1996-07-15,7.0



--- Verifying the data type of the new column ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   customer_id          2155 non-null   object        
 1   city                 2155 non-null   object        
 2   order_id             2155 non-null   int64         
 3   order_date           2155 non-null   datetime64[ns]
 4   shipped_date         2082 non-null   datetime64[ns]
 5   product_name         2155 non-null   object        
 6   category_name        2155 non-null   object        
 7   quantity             2155 non-null   int64         
 8   unit_price           2155 non-null   float64       
 9   discount             2155 non-null   float64       
 10  estimated_item_cost  2155 non-null   float64       
 11  item_gross_revenue   2155 non-null   float64       
 12  item_total_cost      2155 non-null   fl